This tutorial demonstrates how to get started with JAX transformations in cryoJAX.

In [ ]:
# Start by creating an image model
import cryojax.simulator as cxs
from cryojax.io import read_array_from_mrc


# Scattering potential stored in MRC format
filename = "./data/groel_5w0s_scattering_potential.mrc"
# ... read into a FourierVoxelGridPotential
real_voxel_grid, voxel_size = read_array_from_mrc(filename, loads_spacing=True)
potential = cxs.FourierVoxelGridPotential.from_real_voxel_grid(
    real_voxel_grid, voxel_size, pad_scale=2
)
# Now, instantiate the pose. Angles are given in degrees
pose = cxs.EulerAnglePose(
    offset_x_in_angstroms=5.0,
    offset_y_in_angstroms=-3.0,
    phi_angle=20.0,
    theta_angle=80.0,
    psi_angle=-5.0,
)
# First, the contrast transfer theory
ctf = cxs.CTF(
    defocus_in_angstroms=10000.0,
    astigmatism_in_angstroms=-100.0,
    astigmatism_angle=10.0,
)
transfer_theory = cxs.ContrastTransferTheory(ctf, amplitude_contrast_ratio=0.1)
# Then the configuration. Add padding with respect to the final image shape.
pad_options = dict(shape=potential.shape[0:2])
config = cxs.BasicConfig(
    shape=(80, 80),
    pixel_size=potential.voxel_size,
    voltage_in_kilovolts=300.0,
    pad_options=pad_options,
)
# Make the image model as well as a simulation function. By default, cryoJAX will simulate
# the contrast in physical units. Rather, normalize the image.
image_model, simulate_fn = cxs.make_image_model(
    potential,
    config,
    pose,
    transfer_theory,
    normalizes_signal=True,
)

In the [Simulate an image](https://mjo22.github.io/cryojax/examples/simulate-image/) tutorial we showed how to simulate an image with the syntax

```python
image_model, simulate_fn = cxs.make_image_model(...)
image = simulate_fn(image_model)
```

In this tutorial, we will show how to perform [transformations](https://docs.jax.dev/en/latest/key-concepts.html#transformations) of `simulate_fn` using `jax.jit`, `jax.grad`, and `jax.vmap`. To do this, we will leverage [filtered transformations](https://docs.kidger.site/equinox/all-of-equinox/#2-filtering) in `equinox`.